In [1]:
import sys
#sys.path.append('/content/drive/My Drive/APT/newest/TAR-project-master')
sys.path.append('/Users/patrik/PycharmProjects/TAR-project')

In [2]:
import pandas as pd
import numpy as np
import nltk

nltk.download('stopwords')
nltk.download('punkt')

from dataset.preprocessing.preprocessing import word_stem

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/patrik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/patrik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sentence_transformers import SentenceTransformer
from query_model.query_engines import BOWQueryEngine, W2VQueryEngine, D2VQueryEngine, BERTQueryEngine
from query_model.transformers.bm25 import BM25Transformer

In [4]:
corpus_path = '/Users/patrik/PycharmProjects/TAR-project/data/corpus.csv'

corpus = pd.read_csv(corpus_path).drop(columns='Unnamed: 0')
corpus.head()

,paper_id,section,text,preprocessed_text
0,a5729379cb2d18730da32cf117283be2cc16cf05,"Dear Editor,","In December 2019, a novel human coronavirus ca...",decemb two thousand and nineteen novel human c...
1,a5729379cb2d18730da32cf117283be2cc16cf05,"Dear Editor,",The nomenclature for this coronavirus is still...,the nomenclatur coronaviru still controversi ....
2,a5729379cb2d18730da32cf117283be2cc16cf05,"Dear Editor,",Coronaviruses are enveloped non-segmented posi...,coronavirus envelop non-seg posit sens rna vir...
3,a5729379cb2d18730da32cf117283be2cc16cf05,"Dear Editor,",Receptor recognition is an important factor de...,receptor recognit import factor determin host ...
4,a5729379cb2d18730da32cf117283be2cc16cf05,"Dear Editor,","In this study, we discovered one deletion and ...",studi discov one delet three insert 2019-ncov ...


## QUERIES

In [4]:
queries = [
    'Smoking effect on the severity of covid19',
    'Incubation period of covid19',
    'Covid19 effect on pregnancy',
    'Mortality rate of covid19',
    'Comorbidities and coinfections with effect on the severity of covid19',
    'Temperature effect on covid19',
    'Ways of covid19 transmission',
    'Basic reproductive number of covid19'
]

color_list = ['black', 'red', 'blue', 'green']
query_colors = {}
for i in range(len(queries)):
    query_colors[queries[i]] = color_list[i % len(color_list)]


def run_queries(query_engine, queries, k=5):
    results = query_engine.run_query(queries[0], k)
    if len(queries) > 1:
        for q in queries[1:]:
            results = pd.concat([results, query_engine.run_query(q, k)])
    return results

def color_rows(val):
    col = colors[val['query']]
    return ['color: %s' % col]*len(val)

## MODELS

In [6]:
# TODO: train and save tf-idf and bm25
stop_words = set(stopwords.words('english'))
cv = CountVectorizer(stop_words=stop_words)
transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
query_engine = BOWQueryEngine(cv, transformer)
query_engine.fit(corpus)

In [7]:
query_engine.save('/Users/patrik/PycharmProjects/TAR-project/trained_models/', 'tf_idf')

Writing object to /Users/patrik/PycharmProjects/TAR-project/trained_models/tf_idf.dat


In [8]:
stop_words = set(stopwords.words('english'))
cv = CountVectorizer(stop_words=None)
transformer = BM25Transformer()
query_engine = BOWQueryEngine(cv, transformer)
query_engine.fit(corpus)

In [9]:
query_engine.save('/Users/patrik/PycharmProjects/TAR-project/trained_models/', 'bm25')

Writing object to /Users/patrik/PycharmProjects/TAR-project/trained_models/bm25.dat


In [10]:
tf_idf = BOWQueryEngine.load('/Users/patrik/PycharmProjects/TAR-project/trained_models/tf_idf.dat')
bm25 = BOWQueryEngine.load('/Users/patrik/PycharmProjects/TAR-project/trained_models/bm25.dat')
w2v = W2VQueryEngine.load('/Users/patrik/PycharmProjects/TAR-project/trained_models/w2v.dat')
w2v_tf_idf = W2VQueryEngine.load('/Users/patrik/PycharmProjects/TAR-project/trained_models/w2vtfidf.dat')
d2v = D2VQueryEngine.load('/Users/patrik/PycharmProjects/TAR-project/trained_models/d2v1.dat')

/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.21.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.21.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [ ]:
models = [tf_idf, bm25, w2v, w2v_tf_idf, d2v]
model_names = ['tf_idf', 'bm25', 'w2v', 'w2v_tf_idf', 'd2v']
model_colors = {}
for i in range(len(model_names)):
    model_colors[model_names[i]] = color_list[i % len(color_list)]

def color_rows(val):
    col = model_colors[val['model']]
    return ['color: %s' % col]*len(val)

def run_query_on_models(query, models, model_names, k=3):
    results = []
    for i in range(len(models)):
        res = models[i].run_query(query, k)
        res['model'] = model_names[i]
        results.append(res)
    return pd.concat(results).reset_index()[['query', 'text', 'model', 'sim']]

In [26]:
# we had some problems with saving the model that uses BERT,
# so we saved the results for the next 8 queries, 
# this is something we should fix in the future
bert_results_path = '/Users/patrik/PycharmProjects/TAR-project/data/roBERTa_results.csv'
bert_results = pd.read_csv(bert_results_path)
bert_results['model'] = 'SBERT'
bert_results = bert_results[['query', 'text', 'model', 'sim']]

### 'Smoking effect on the severity of covid19'

In [28]:
smoking_query = 'Smoking effect on the severity of covid19'

In [14]:
smoking_res = run_query_on_models(smoking_query, models, model_names)
smoking_res.style.set_properties(**{'font-size': '9pt'}).apply(color_rows, axis=1)

100%|██████████| 1/1 [00:00<00:00, 761.63it/s]


In [34]:
bert_results[bert_results['query']==smoking_query][:3].style.set_properties(**{'font-size': '9pt'})

,query,text,model,sim
50,Smoking effect on the severity of covid19,"Smoking is associated with COVID-19 disease progression. Physicians and public health professionals should collect data on smoking and, given the pulmonary effects of e-cigarettes, 3 e-All rights reserved. No reuse allowed without permission.",SBERT,0.885712
51,Smoking effect on the severity of covid19,"The studies analyzed are presented in Table 1 . A total of 5960 hospitalized COVID-19 patients from China were presented in all studies. The majority (55.1%) were males. The mean or median age ranged from 38 to 68 years, with some studies presenting mean age in different subgroups. One study presented patients divided into two subgroups according to age (< 60 years and ≥ 60 years) [16] while another one reported cases according to the presence or absence of gastrointestinal symptoms. [17] In both studies, the 2 subgroups were combined and treated as one sample. Two studies reported the smoking status as ""history of smoking"" or ""smoking"". [23, 25] Although it was unclear if this included both current and former smokers, we analyzed the data assuming they were all current smokers. In most studies, former smokers were not reported ( Table 1) . The current smoking prevalence ranged from 1.4 to 12.6%. The random effects pooled prevalence of current smoking was 6.5% (95% CI 4.9-8.2%). Findings from the random-effects meta-analysis are presented in Fig. 1 . The observed outcome is the estimated prevalence of current smoking among those who were hospitalized with COVID-19. The forest plot depicts the fractional prevalence of smokers in each study. The size of the dark squares is proportional to the weight each study has in determining the combined effect size estimate. The combined effect estimate is depicted via the dark diamond (RE Model), with the 95% confidence interval corresponding to the width of the diamond. The meta-analysis also indicated statistically significant heterogeneity between studies (p < 0.001).",SBERT,0.866208
52,Smoking effect on the severity of covid19,"The current smoking prevalence ranged from 1.4 to 12.6%. The random effects pooled prevalence of current smoking was 6.5% (95% CI 4.9–8.2%). Findings from the random-effects meta-analysis are presented in Fig. 1. The observed outcome is the estimated prevalence of current smoking among those who were hospitalized with COVID-19. The forest plot depicts the fractional prevalence of smokers in each study. The size of the dark squares is proportional to the weight each study has in determining the combined effect size estimate. The combined effect estimate is depicted via the dark diamond (RE Model), with the 95% confidence interval corresponding to the width of the diamond. The meta-analysis also indicated statistically significant heterogeneity between studies (p < 0.001).",SBERT,0.866208


### 'Incubation period of covid19'

In [48]:
incub_query = 'Incubation period of covid19'

In [15]:
incub_res = run_query_on_models(incub_query, models, model_names)
incub_res.style.set_properties(**{'font-size': '9pt'}).apply(color_rows, axis=1)

100%|██████████| 1/1 [00:00<00:00, 537.11it/s]


,query,text,model,sim
0,Incubation period of covid19,"At first glance, the predicted numbers of preventable infections seem surprisingly high compared to the confirmed number of 769 positive COVID19 cases in WA 3 and 4661 confirmed positive cases in the entire US. However, the growth rate of COVID19 in the US has been rapid, increasing from 69 cases on 2/29/20 to 4661 confirmed cases just 17 days later. 2 Due to the current state of COVID19 testing, US statistics on confirmed COVID19 cases may not be the most reliable, either. Per CDC guidelines that were last updated 3/9/2020, laboratory testing for COVID19 is only indicated for individuals who both develop respiratory symptoms consistent with COVID19 and meet additional criteria, such as being hospitalized, having certain comorbidities, and/or having contact with suspected COVID19+ individuals. 25 However, many COVID19+ individuals may be asymptomatic or only have mild symptoms. 26 In addition, COVID19 testing shortages may make the US statistics on COVID19 cases less reliable. 27 According to Dr.",tf_idf,0.546616
1,Incubation period of covid19,"The incubation period is essential in the control of infectious diseases. To halt a pandemic, a suitable quarantine period should be set by investigating incubation periods. Thus, understanding the characteristics of the incubation period is crucial to design public health efforts. In this cohort, the full range of incubation periods of the Covid-19 cases ranged from 0 to 33 days among 2015 cases. Both male and female adults had their median incubation periods of 7-day, which is 1.8-day longer than those in other reports 3,4 (see discussion). However, a similar result was released in the recent news from the Chinese Medical Association that the median incubation periods were 5-7 days 5 . The median incubation period (9-day) of children were significantly longer than those of adults (t-test P = 0.02). For adults, the incubation periods for the majority (66.5%) of the cases were 1-9 days: most of them (57.6% and 52.0% for males and females, respectively) had 1-5 days incubation.",tf_idf,0.544855
2,Incubation period of covid19,"In addition, we considered COVID-19 to be infectious in the later stage of the incubation period: there is no evidence of weaker infectivity during the incubation period, and it is assumed that it can be as infectious as during the infected period, indicating "" = ! = 2%. The incubation period is generally 2-14 days, and 3-7 days in most cases. The median incubation period is 4 days. Assuming infectivity in the 1-3 days before the end of the incubation period, with an average of 2 days, the ratio of infectious exposed people ( "" ) can be calculated as follows",tf_idf,0.534994
3,Incubation period of covid19,Index patient was defined as the first person in a household diagnosed with Covid19 using RT-PCR. The incubation period was calculated from the time between the last potential exposure and the time showing the first disease symptoms.,bm25,0.491030
4,Incubation period of covid19,"The copyright holder for this preprint this version posted March 20, 2020. Due to the 14-day incubation period of the virus, 6 coupled with the current resource limitations in the US, COVID19 infections will not be detected until symptoms become evident. To estimate the asymptomatic infected population, we looked at publicly available data with at least 15 days of significant increase in confirmed COVID19 case numbers and back-calculated the population count that would have likely been in the pre-symptomatic incubation phase on previous dates. This means that, for any time t, the number of asymptomatic but infected individuals can be estimated using the sum of new infections that were confirmed on t + 1 to t + 14 as follows:",bm25,0.385135
5,Incubation period of covid19,with Males having a 3-fold higher risk of mortality in COVID19 compared to females. 132 133,bm25,0.373722
6,Incubation period of covid19,"In addition, we considered COV

In [49]:
bert_results[bert_results['query']==incub_query][:3].style.set_properties(**{'font-size': '9pt'})

,query,text,model,sim
30,Incubation period of covid19,"Incubation period was calculated using the data of 31 cases with clear cutting time points of exposure and illness onset. The mean incubation period of COVID-19 was . CC-BY-NC-ND 4.0 International license It is made available under a is the author/funder, who has granted medRxiv a license to display the preprint in perpetuity.",SBERT,0.880740
31,Incubation period of covid19,"We conducted nucleic acid testing for a total of 2004 people and three of these in our study tested positive for 2019nCoV. After recheck by Shenzhen Centers for Disease Control and Prevention, these persons were confirmed as COVID-19 cases. The incidence of COVID-19",SBERT,0.856750
32,Incubation period of covid19,All palliative treatments were and will be delivered through a hypofractionationed regimen (Either 5 or 1 fraction) until the outbreak is over. Breast cancer adjuvant radiotherapy treatement is one example planned to be delivered through a hypofractionationed regimen. At the time of COVID-19,SBERT,0.850843


### 'Covid19 effect on pregnancy'

In [35]:
pregn_query = 'Covid19 effect on pregnancy'

In [16]:
pregn_res = run_query_on_models(pregn_query, models, model_names)
pregn_res.style.set_properties(**{'font-size': '9pt'}).apply(color_rows, axis=1)

100%|██████████| 1/1 [00:00<00:00, 972.71it/s]


,query,text,model,sim
0,Covid19 effect on pregnancy,with confirmed COVID-19 infection in pregnancy. 23,tf_idf,0.556460
1,Covid19 effect on pregnancy,"At first glance, the predicted numbers of preventable infections seem surprisingly high compared to the confirmed number of 769 positive COVID19 cases in WA 3 and 4661 confirmed positive cases in the entire US. However, the growth rate of COVID19 in the US has been rapid, increasing from 69 cases on 2/29/20 to 4661 confirmed cases just 17 days later. 2 Due to the current state of COVID19 testing, US statistics on confirmed COVID19 cases may not be the most reliable, either. Per CDC guidelines that were last updated 3/9/2020, laboratory testing for COVID19 is only indicated for individuals who both develop respiratory symptoms consistent with COVID19 and meet additional criteria, such as being hospitalized, having certain comorbidities, and/or having contact with suspected COVID19+ individuals. 25 However, many COVID19+ individuals may be asymptomatic or only have mild symptoms. 26 In addition, COVID19 testing shortages may make the US statistics on COVID19 cases less reliable. 27 According to Dr.",tf_idf,0.514736
2,Covid19 effect on pregnancy,"The recent outbreak of Covid19 has required urgent treatments for numerous patients. No suitable vaccines or antivirals are available for Covid19. The efficiency against Covid19 of WHO therapies of choice, that are two antivirals developed for other pathologies, is controversial. Therefore, alternative approaches are required. Intravenous (IV)",tf_idf,0.434348
3,Covid19 effect on pregnancy,with confirmed COVID-19 infection in pregnancy. 23,bm25,0.655885
4,Covid19 effect on pregnancy,"We focused on the pregnant COVID19 patients. Given the maternal physiologic and immune function changes in pregnancy [2] , pregnant individuals might face greater risk of getting infected by SARS-CoV-2 and might have more complicated clinical events. We described epidemiological, clinical characteristics, pregnancy and perinatal outcomes of all hospitalized pregnant patients diagnosed with COVID-19 in China.",bm25,0.406928
5,Covid19 effect on pregnancy,4. There are repeated waves of COVID19 until an effective vaccine is employed.,bm25,0.366041
6,Covid19 effect on pregnancy,"• In pregnancy complicated by SARS-CoV-2, antenatal corticosteroids should be given with caution • When needed, antenatal low-dose (versus full-dose) corticosteroids can still be beneficial, with minimal effect on glycemia 3 Abstract Background and Aims: Administration of corticosteroids is common in obstetric practice. In this concise review we queried on the effects of corticosteroids in pregnancies complicated by SARS-CoV-2. We performed a literature search on PubMed, regarding the use of corticosteroids in patients with SARS-CoV-2 infection, in pregnancies complicated by SARS-CoV-2, as well as their impact on glycemia in pregnant women with or without diabetes. Furthermore, we searched for effects of SARS-CoV-2 and of other coronaviridae on insulin secretion and glycemia. Results: SARS-CoV-2 infection appears to be a risk factor for complications in pregnancy. Corticosteroids may not be recommended for treating SARS-CoV-2 pneumonia but they may be needed for at-risk pregnancies. Corticosteroids in pregnancy have a diabetogenic potential. SARS-CoV-2 and other coronaviridae may have effects on glycemia. Conclusions: Caution should be exercised while using corticosteroids in pregnant women with COVID-19 requiring preterm delivery",w2v,0.758627
7,Covid19 effect on pregnancy,"Early detection and intervention of COVID-19 may reduce potential obstetrical complications such as pregnancy loss, intrauterine growth restriction, and preterm delivery, and may be beneficial for improving pregnancy outcomes. Anti-viral therapy for COVID-19, such as lopinavir and ritonavir, should be determined by weighing the risks and benefits. Treatment can be initiated when the potential benefits outweigh the potent

In [36]:
bert_results[bert_results['query']==pregn_query][:3].style.set_properties(**{'font-size': '9pt'})

,query,text,model,sim
20,Covid19 effect on pregnancy,"A review of prior and the novel coronaviruses and their impact on human reproduction, specifically male and female gametes, pregnancy, and postpartum suggests that much remains unknown. Objective: To summarize current understanding of the effects of novel and prior coronaviruses on human reproduction, specifically male and female gametes, and in pregnancy. Design: Review of English publications in PubMed and Embase to April 6, 2020. Methods: Manuscripts were screened for reports including coronavirus, reproduction, including pathophysiology and pregnancy. Main Outcome Measure(s): Reproductive outcomes; effects on gametes; pregnancy outcomes; neonatal complications. Results: Seventy-nine reports formed the basis of the review. Coronavirus binding to cells involves the S1 domain of the spike protein to receptors present in reproductive tissues, including angiotensin converting enzyme-2 (ACE2), CD26, Ezrin, and cyclophilins. SARS-CoV-1 may cause severe orchitis leading to germ cell destruction in males. Reports indicate decreased sperm concentration and motility for 72-90 days following COVID-19 infection. Gonadotropindependent expression of ACE2 was found in human ovaries, but it is unclear whether SARS-CoV-2 adversely affects female gametogenesis. Evidence suggests that COVID-19 infection has a lower maternal case fatality rate than SARS or MERS, but anecdotal reports suggest that infected, asymptomatic women may develop respiratory symptoms postpartum. COVID-19 infections in pregnancy are associated with preterm delivery. Postpartum neonatal transmission from mother to child has been reported. Conclusion: COVID-19 infection may adversely affect some pregnant women and their offspring. Additional studies are needed to assess effects of SARS-CoV-2 infection on male and female fertility.",SBERT,0.827413
21,Covid19 effect on pregnancy,"The emergence of the Middle East Respiratory Syndrome Coronavirus (MERS-CoV) in 2012 was associated with variable rate of asymptomatic infection but no asymptomatic cases were reported among pregnancy [1]. The newly emerging 2019 novel coronavirus (2019-nCoV), later named SARS-CoV-2 is the causative agent of the COVID-19. SARS-CoV-2 was identified initially in Wuhan, China. Since its first description in December 2019, the total number of cases as reported by the World Health Organization (WHO) had reached 191,127confirmed cases on February 18, 2020, with 7807 deaths [2]. The clinical picture and radiographic presentations were recently described [3,4] and the clinical picture in nine pregnant women [5] were similar to non-pregnant women [3,4]. One study showed that 15 pregnant patients with COVID-19 had mild disease [6]. All of the pregnant patients in one study and 66.7% of pregnant COVID-19 patients had C-section, moreover all the newborn survived [5,6]. However, 4 of 9 (44%) pregnant COVID-19 patients had premature delivery [5]. In a previous study of MERS-CoV, there were 11 pregnant women [7]. A comparison between MERS-CoV and COVID-19 cases in pregnancy is shown in Table 1 . There was no difference in the age group of the patients, however, the gestational age was lower among MERS-CoV than COVID-19 patients. Of the MERS-CoV cases in pregnancy, 63.6% required intensive care unit admission and this is comparable to 50% of SARS pregnant women [8]. There was no mention about the death rate among pregnant women with COVID-19, however, the case fatality rate among MERS-pregnant patients was about 35% and was not statistically different when compared to the overall MERS case fatality rate [7]. In the MERS cases, 40% had C-section and this is much lower than 100% C-section rate in the case of COVID-19. This difference may reflect a variance in the practices between different countries. Another difference is the high fetal demise rate of 30% among pregnant women with MERS compared to 0% among COVID-19. The fetal demise rate among 12 pregnant women w

### 'Mortality rate of covid19'

In [37]:
mort_query = 'Mortality rate of covid19'

In [17]:
mort_res = run_query_on_models(mort_query, models, model_names)
mort_res.style.set_properties(**{'font-size': '9pt'}).apply(color_rows, axis=1)

100%|██████████| 1/1 [00:00<00:00, 895.26it/s]


,query,text,model,sim
0,Mortality rate of covid19,"At first glance, the predicted numbers of preventable infections seem surprisingly high compared to the confirmed number of 769 positive COVID19 cases in WA 3 and 4661 confirmed positive cases in the entire US. However, the growth rate of COVID19 in the US has been rapid, increasing from 69 cases on 2/29/20 to 4661 confirmed cases just 17 days later. 2 Due to the current state of COVID19 testing, US statistics on confirmed COVID19 cases may not be the most reliable, either. Per CDC guidelines that were last updated 3/9/2020, laboratory testing for COVID19 is only indicated for individuals who both develop respiratory symptoms consistent with COVID19 and meet additional criteria, such as being hospitalized, having certain comorbidities, and/or having contact with suspected COVID19+ individuals. 25 However, many COVID19+ individuals may be asymptomatic or only have mild symptoms. 26 In addition, COVID19 testing shortages may make the US statistics on COVID19 cases less reliable. 27 According to Dr.",tf_idf,0.625122
1,Mortality rate of covid19,"The recent outbreak of Covid19 has required urgent treatments for numerous patients. No suitable vaccines or antivirals are available for Covid19. The efficiency against Covid19 of WHO therapies of choice, that are two antivirals developed for other pathologies, is controversial. Therefore, alternative approaches are required. Intravenous (IV)",tf_idf,0.513237
2,Mortality rate of covid19,with Males having a 3-fold higher risk of mortality in COVID19 compared to females. 132 133,tf_idf,0.505090
3,Mortality rate of covid19,with Males having a 3-fold higher risk of mortality in COVID19 compared to females. 132 133,bm25,0.537553
4,Mortality rate of covid19,Thermal screening at airport has 54% COVID19 detection rate [1] . 72% with COVID19 infection have their sputum tested positive [2] . 3.5% individuals with sputum positivity will not have disease [3] .,bm25,0.436821
5,Mortality rate of covid19,"• #VIRUS: Hashtags which relate only to Corona Virus, e.g. #coronavirus or #covid19.",bm25,0.421063
6,Mortality rate of covid19,"The current meta-analysis provided more information on incidence, mortality, and recovery rates between COVID-19 and SARS-CoV infectious diseases. Based on event rate evaluation, the incidence rate of COVID-19 was higher than SARS-CoV's where mortality and recovery rates for SARS-CoV were higher in comparison with COVID-19. On the other hand, the risk ratio and subgroup analysis of ''incidence vs. mortality'' and ''recovery vs. mortality'' for both COVID-19 and SARS-CoV revealed new findings. Those were the higher mortality and recovery rates of both SARS-CoV and COVID-19 in comparison to incidence and mortality rates, respectively.",w2v,0.869497
7,Mortality rate of covid19,"And, the mortality rate is higher than the incidence rate in both infection groups, and subgroup analysis showed that in SARS infection, the mortality rate is more than incidence rate in comparison to COVID-19 disease ( p !0.001). Cumulative incidence vs. mortality rateof COVID-19 and SARS-CoV infections reflected the above results with p !0.001.",w2v,0.864807
8,Mortality rate of covid19,"Which resulted, the recovery rate is more than the mortality rate in both infectious groups, And subgroup analysis has shown that in SARS infection, the recovery rate vs. mortality rate is higher than that of COVID-19 (p <0.008). And, cumulative recovery rate vs. mortality rate of COVID-19 and SARS-CoV infections reflected the above results with p <0.001.",w2v,0.851879
9,Mortality rate of covid19,"The current meta-analysis provided more information on incidence, mortality, and recovery rates between COVID-19 and SARS-CoV infectious diseases. Based on event rate evaluation, the incidence rate of COVID-19 was higher than SARS-CoV's where mortality and recovery rates for SARS-CoV were higher in comparison with COVID-19. On the other hand, the risk ratio and subgroup analysi

In [38]:
bert_results[bert_results['query']==mort_query][:3].style.set_properties(**{'font-size': '9pt'})

,query,text,model,sim
40,Mortality rate of covid19,"The Coronaviridae family of viruses includes six prev iously known humaninfecting coronaviruses, which are enve loped, nonsegmented, positivesense RNA viruses that are broadly distributed in humans and other mammals. 13 Although four of these coronaviruses cause only mild respiratory symptoms similar to the common cold, epi demics caused by two beta coronaviruses, SARS CoV in 2002 14, 15 and MERSCoV in 2012, involved more than 8000 patients and 1000 patients, respectively, 16, 17 with high mortality rates (10% for SARSCoV and 37% for MERSCoV). The novel coronavirus SARSCoV2 is the seventh member of the Coronaviridae family known to infect humans. The mortality rate of COVID19 so far is",SBERT,0.880951
41,Mortality rate of covid19,"The most common complications that develop in COVID-19 are bilateral pneumonia which may progress to ARDS, sepsis and septic shock, acute kidney injury (AKI) and others such as acute cardiac injury (arrhythmias, heart failure, MI), coagulopathy, rhabdomyolysis, hyponatremia and acidosis. Complications are more in severe disease vs. non-severe disease. In the study cohort of 1099 COVID-19 cases by Guan et al. (2020), bilateral pneumonia (91.1%) occurred most commonly followed by ARDS (3.4%), septic shock (1.1%), AKI (0.5%) and coagulopathy (0.1%). In this cohort 926 (84%) had non-severe and 173 (16%) had severe disease. Underlying co-morbidities were more common among patients with severe disease than among non-severe disease (38.7% vs. 21.0%). Outcomes were as follows: 55 (5%) were discharged, 15 (1.4%) died, 9 (0.8%) recovered and 1029 (93.6%) remain hospitalized as of March 6, 2020. Overall mortality associated with COVID as of March 1, 2020, as estimated by the WHO, is 3.6% in China and 1.5% outside China. The mortality of COVID-19 (Fig. 6.4) is much less than that of SARS (9.6%) and MERS (34%). Case fatality rate of COVID-19 ranges from 5.8% in Wuhan to 0.7% in China. It is higher in elderly >70 years (8–15%). Overall case fatality is 2.3% (WHO 2020c). In most of the people with COVID-19, almost >80% have mild illness (Wu and McGoogan 2020).",SBERT,0.875505
42,Mortality rate of covid19,"The most common complications that develop in COVID-19 are bilateral pneumonia which may progress to ARDS, sepsis and septic shock, acute kidney injury (AKI) and others such as acute cardiac injury (arrhythmias, heart failure, MI), coagulopathy, rhabdomyolysis, hyponatremia and acidosis. Complications are more in severe disease vs. non-severe disease. In the study cohort of 1099 COVID-19 cases by Guan et al. (2020) , bilateral pneumonia (91.1%) occurred non-survivors most commonly followed by ARDS (3.4%), septic shock (1.1%), AKI (0.5%) and coagulopathy (0.1%). In this cohort 926 (84%) had non-severe and 173 (16%) had severe disease. Underlying co-morbidities were more common among patients with severe disease than among non-severe disease (38.7% vs. 21.0%). Outcomes were as follows: 55 (5%) were discharged, 15 (1.4%) died, 9 (0.8%) recovered and 1029 (93.6%) remain hospitalized as of March 6, 2020. Overall mortality associated with COVID as of March 1, 2020, as estimated by the WHO, is 3.6% in China and 1.5% outside China. The mortality of COVID-19 ( Fig. 6.4) is much less than that of SARS (9.6%) and MERS (34%). Case fatality rate of COVID-19 ranges from 5.8% in Wuhan to 0.7% in China. It is higher in elderly >70 years (8-15%). Overall case fatality is 2.3% (WHO 2020c). In most of the people with COVID-19, almost >80% have mild illness (Wu and McGoogan 2020).",SBERT,0.843248


### 'Comorbidities and coinfections with effect on the severity of covid19'

In [39]:
coco_query = 'Comorbidities and coinfections with effect on the severity of covid19'

In [18]:
coco_res = run_query_on_models(coco_query, models, model_names)
coco_res.style.set_properties(**{'font-size': '9pt'}).apply(color_rows, axis=1)

100%|██████████| 1/1 [00:00<00:00, 515.21it/s]


,query,text,model,sim
0,Comorbidities and coinfections with effect on the severity of covid19,"At first glance, the predicted numbers of preventable infections seem surprisingly high compared to the confirmed number of 769 positive COVID19 cases in WA 3 and 4661 confirmed positive cases in the entire US. However, the growth rate of COVID19 in the US has been rapid, increasing from 69 cases on 2/29/20 to 4661 confirmed cases just 17 days later. 2 Due to the current state of COVID19 testing, US statistics on confirmed COVID19 cases may not be the most reliable, either. Per CDC guidelines that were last updated 3/9/2020, laboratory testing for COVID19 is only indicated for individuals who both develop respiratory symptoms consistent with COVID19 and meet additional criteria, such as being hospitalized, having certain comorbidities, and/or having contact with suspected COVID19+ individuals. 25 However, many COVID19+ individuals may be asymptomatic or only have mild symptoms. 26 In addition, COVID19 testing shortages may make the US statistics on COVID19 cases less reliable. 27 According to Dr.",tf_idf,0.472500
1,Comorbidities and coinfections with effect on the severity of covid19,Coronavirus in only one case. Two among 4 deaths had coinfection with rhinovirus. Viral coinfection was not related to mortality in our study.,tf_idf,0.463116
2,Comorbidities and coinfections with effect on the severity of covid19,"Coinfection with HCoVs and other respiratory viruses is frequently observed and is associated with severe clinical syndromes, especially in infants and young children (6, 8) . Coinfection was observed in 3 of the 7 HCoV-positive patients in our study. The 3 patients were infected with HCoV-OC43 or HCoV-229E; coinfection with respiratory syncytial virus was found in 2 patients, and coinfection with Mycoplasma pneumoniae was found in 1 patient. It was not possible to determine the role of the HCoVs in these coinfections. In addition, because coronaviruses can be detected even 3 weeks after an acute episode, some cases of coinfection might represent former rather than current HCoV infection (9) .",tf_idf,0.422091
3,Comorbidities and coinfections with effect on the severity of covid19,Coronavirus in only one case. Two among 4 deaths had coinfection with rhinovirus. Viral coinfection was not related to mortality in our study.,bm25,0.452744
4,Comorbidities and coinfections with effect on the severity of covid19,Viral studies were available in 15 subjects (88%). Eight patients (53%) had viral coinfection with Rhinovirus in 7 cases and with Rhinovirus and Coronavirus in only one case. Two among 4 deaths had coinfection with rhinovirus. Viral coinfection was not related to mortality in our study.,bm25,0.383275
5,Comorbidities and coinfections with effect on the severity of covid19,4. There are repeated waves of COVID19 until an effective vaccine is employed.,bm25,0.321219
6,Comorbidities and coinfections with effect on the severity of covid19,". [1] Metabolic and cardiovascular comorbidities like diabetes and hypertension aggravate the severity of COVID-19. [2] Another comorbidity, MAFLD, also affects COVID-19 severity, as pointed out by Ji et al.. Since excess liver fat is seen in up to a quarter of people, [3] we hypothesized that its impact on severity might be modulated by age. We considered that disease severity of older patients with a greater burden of cardiac and respiratory illness would more likely be impacted by their comorbid conditions, than the presence of liver fat. In this study, we investigated the effects of MAFLD on COVID-19 severity in older versus younger patients.",w2v,0.829363
7,Comorbidities and coinfections with effect on the severity of covid19,"The findings reported here are in keeping with current knowledge that the elderly and those with comorbidities are more susceptible to severe infection. those with: cardiovascular disease (10.5%), diabetes (7.3%), chronic respiratory disease (6.3%) and hypertension (6.

In [40]:
bert_results[bert_results['query']==coco_query][:3].style.set_properties(**{'font-size': '9pt'})

,query,text,model,sim
10,Comorbidities and coinfections with effect on the severity of covid19,"COVID-19 is a global disease which is severely testing the health systems of many countries. Evidence is currently lacking for the correct management of patients. In many cases, comorbidities and coinfections coexist, complicating the clinical scenario, in particular in older people who have the worst outcomes for COVID-19. We hope that further studies will reveal new treatment opportunities to better manage these patients.",SBERT,0.749451
11,Comorbidities and coinfections with effect on the severity of covid19,"The copyright holder for this preprint this version posted April 24, 2020. In addition to the clinical criteria, it is of great benefit to have laboratory markers for severe infection which will help in monitoring and prognosis and will be used to develop workup and treatment algorithms. This meta-analysis investigated the association between severe COVID-19",SBERT,0.743938
12,Comorbidities and coinfections with effect on the severity of covid19,"The analysis of medications acting on renin-angiotensin-aldosterone system demonstrates that patients with COVID-19 were significantly less likely compared to population-based controls to be taking ARBs (8.7% vs 15.4%) and diuretics in (2.1% vs 5.1%) prior to their presentation, after adjusting for covariates such as age, sex and BMI) as well as presence of hypertension. In contrast, CCBs were associated with increased risk of COVID-19. Although hypertension is clearly a risk factor for severity, is the author/funder, who has granted medRxiv a license to display the preprint in perpetuity.",SBERT,0.730677


### 'Temperature effect on covid19'

In [41]:
temp_query = 'Temperature effect on covid19'

In [19]:
temp_res = run_query_on_models(temp_query, models, model_names)
temp_res.style.set_properties(**{'font-size': '9pt'}).apply(color_rows, axis=1)

100%|██████████| 1/1 [00:00<00:00, 472.86it/s]


,query,text,model,sim
0,Temperature effect on covid19,"At first glance, the predicted numbers of preventable infections seem surprisingly high compared to the confirmed number of 769 positive COVID19 cases in WA 3 and 4661 confirmed positive cases in the entire US. However, the growth rate of COVID19 in the US has been rapid, increasing from 69 cases on 2/29/20 to 4661 confirmed cases just 17 days later. 2 Due to the current state of COVID19 testing, US statistics on confirmed COVID19 cases may not be the most reliable, either. Per CDC guidelines that were last updated 3/9/2020, laboratory testing for COVID19 is only indicated for individuals who both develop respiratory symptoms consistent with COVID19 and meet additional criteria, such as being hospitalized, having certain comorbidities, and/or having contact with suspected COVID19+ individuals. 25 However, many COVID19+ individuals may be asymptomatic or only have mild symptoms. 26 In addition, COVID19 testing shortages may make the US statistics on COVID19 cases less reliable. 27 According to Dr.",tf_idf,0.568303
1,Temperature effect on covid19,"The recent outbreak of Covid19 has required urgent treatments for numerous patients. No suitable vaccines or antivirals are available for Covid19. The efficiency against Covid19 of WHO therapies of choice, that are two antivirals developed for other pathologies, is controversial. Therefore, alternative approaches are required. Intravenous (IV)",tf_idf,0.479549
2,Temperature effect on covid19,"Pre-existing lung disease and diabetes were consistently associated with a higher risk of requiring respiratory support with OR ranging from 1.52 for lung disease in T-COVID19 to 2.03 for diabetes in SR-COVID19. Individuals with obesity were notably more likely to require medical support for the T-COVID19 and I-COVID19 groups with OR of 1. 33 19 , T-COVID-19 and I-COVID-19 respectively. Age was also more predictive of treatment course than of hospital visit with OR around 1.02 in all three groups.",tf_idf,0.458646
3,Temperature effect on covid19,4. There are repeated waves of COVID19 until an effective vaccine is employed.,bm25,0.415551
4,Temperature effect on covid19,with Males having a 3-fold higher risk of mortality in COVID19 compared to females. 132 133,bm25,0.390719
5,Temperature effect on covid19,"• #VIRUS: Hashtags which relate only to Corona Virus, e.g. #coronavirus or #covid19.",bm25,0.385125
6,Temperature effect on covid19,"There is currently debate on the impact of higher summer temperatures of Arizona on the transmission rate of COVID-19, both with regard to a potentially suppressive effect on virus survivability in elevated temperatures, as well as the behavioral effect of extremely hot temperatures changing patterns of indoor and outdoor activity in Arizona's desert environment. While we are not certain about a potential summer effect, particularly on virus survivability, we have found it to be a useful context to demonstrate the sensitivity of outbreak dynamics to the transmission rate β. In Figure 9 , we plotted the total infected and the hospitalizations under four different scenarios with respect to the summer effect. The plot shows the tradeoff between an early summer effect versus a later but more significant summer effect. The figure also demonstrates the impact of a 25% to 50% reduction in the transmission rate as well as the impact of the timing of the summer effect in flattening the curve further.",w2v,0.725944
7,Temperature effect on covid19,"weather alone (i.e., increase of temperature and humidity as spring and summer months arrive in the North Hemisphere) will not necessarily lead to declines in case counts without the implementation of extensive public health interventions. Further studies on the effects of absolute humidity and temperature on COVID-19 transmission are needed.",w2v,0.687792
8,Temperature effect on covid19,"Several studies have demonstrated that there is a correlation between meteorological factors

In [42]:
bert_results[bert_results['query']==temp_query][:3].style.set_properties(**{'font-size': '9pt'})

,query,text,model,sim
60,Temperature effect on covid19,The weather is an important factor in determining the incidence rate of covid-19 in Jakarta. Temperature average was significantly correlated with covid-19. Our findings can be used as an input in suppressing covid-19 disease in Indonesia.,SBERT,0.816446
61,Temperature effect on covid19,"Objective: To estimate the number of deaths in Peru due to COVID-19. Design: With a priori information obtained from the daily number of deaths due to CODIV-19 in China and data from the Peruvian authorities, we constructed a predictive Bayesian non-linear model for the number of deaths in Peru. Exposure: COVID-19. Outcome: Number of deaths. Results: Assuming an intervention level similar to the one implemented in China, the total number of deaths in Peru is expected to be 612 (95%CI: 604.3 -833.7) persons. Sixty four days after the first reported death, the 99% of expected deaths will be observed. The inflexion point in the number of deaths is estimated to be around day 26 (95%CI: 25.1 -26.8) after the first reported death. Conclusion: These estimates can help authorities to monitor the epidemic and implement strategies in order to manage the COVID-19 pandemic.",SBERT,0.799436
62,Temperature effect on covid19,"Literature has considered whether ""black swans"" have a global impact (Wang et al., 2019) . But COVID-19 is a globally impacting phenomenon. There is also the question of whether events of the magnitude of COVID-19 are insurable. Another question is whether COVID-19 should be regarded as a black swan, or an unforeseeable event with extreme consequences. The answer seems to be ""no."" When you have a host of academic articles, discussed above, suggesting the possibility of pandemics and predicting enormous economic losses as a result of pandemics; as well as numerous real-world epidemics and health crises that could have become global pandemics, it should be regarded as something other than totally unexpected.",SBERT,0.798958


### 'Ways of covid19 transmission'

In [44]:
trans_query = 'Ways of covid19 transmission'

In [20]:
trans_res = run_query_on_models(trans_query, models, model_names)
trans_res.style.set_properties(**{'font-size': '9pt'}).apply(color_rows, axis=1)

100%|██████████| 1/1 [00:00<00:00, 444.08it/s]


,query,text,model,sim
0,Ways of covid19 transmission,"At first glance, the predicted numbers of preventable infections seem surprisingly high compared to the confirmed number of 769 positive COVID19 cases in WA 3 and 4661 confirmed positive cases in the entire US. However, the growth rate of COVID19 in the US has been rapid, increasing from 69 cases on 2/29/20 to 4661 confirmed cases just 17 days later. 2 Due to the current state of COVID19 testing, US statistics on confirmed COVID19 cases may not be the most reliable, either. Per CDC guidelines that were last updated 3/9/2020, laboratory testing for COVID19 is only indicated for individuals who both develop respiratory symptoms consistent with COVID19 and meet additional criteria, such as being hospitalized, having certain comorbidities, and/or having contact with suspected COVID19+ individuals. 25 However, many COVID19+ individuals may be asymptomatic or only have mild symptoms. 26 In addition, COVID19 testing shortages may make the US statistics on COVID19 cases less reliable. 27 According to Dr.",tf_idf,0.574540
1,Ways of covid19 transmission,"The recent outbreak of Covid19 has required urgent treatments for numerous patients. No suitable vaccines or antivirals are available for Covid19. The efficiency against Covid19 of WHO therapies of choice, that are two antivirals developed for other pathologies, is controversial. Therefore, alternative approaches are required. Intravenous (IV)",tf_idf,0.484812
2,Ways of covid19 transmission,"The fourth uncertainty is the duration of the infectious period for COVID19. The infectious period is typically short for influenza A, but it seems long for COVID19 on the basis of the few available clinical virological studies, perhaps lasting for 10 days or more after the incubation period. 8 The reports of a few superspreading events are a routine feature of all infectious diseases and should not be overinterpreted. 10 What do these comparisons with influenza A and SARS imply for the COVID19 epidemic and its control? First, we think that the epidemic in any given country will initially spread more slowly than is typical for a new influenza A strain. COVID19 had a doubling time in China of about 4-5 days in the early phases. 3 Second, the COVID19 epidemic could be more drawn out than seasonal influenza A, which has relevance for its potential economic impact. Third, the effect of seasons on transmission of COVID19 is unknown; 11 however, with an R 0 of 2-3, the warm months of summer in the northern hemisphere might not necessarily reduce transmission below the value of unity as they do for influenza A, which typically has an R 0 of around 1·1-1·5. 12 Closely linked to these factors and their epidemiological determinants is the impact of different mitigation policies on the course of the COVID19 epidemic.",tf_idf,0.469145
3,Ways of covid19 transmission,with Males having a 3-fold higher risk of mortality in COVID19 compared to females. 132 133,bm25,0.397235
4,Ways of covid19 transmission,"• #VIRUS: Hashtags which relate only to Corona Virus, e.g. #coronavirus or #covid19.",bm25,0.391548
5,Ways of covid19 transmission,4. There are repeated waves of COVID19 until an effective vaccine is employed.,bm25,0.384551
6,Ways of covid19 transmission,"Indirect transmission via non-contact route can increase the basic reproduction number significantly. Therefore, not incorporating this route of transmission in the model will underestimate the strength of COVID-19 transmission and spread.",w2v,0.815743
7,Ways of covid19 transmission,"2. Airborne transmission and transmission via fomites, which are indirect (not as a consequence of direct person-to-person contact) routes for COVID-19 spread, and which have been recognised as possible causes for infection spread [5, 6] . Strength of the indirect transmission from all external sources (henceforth referred to as 'non-contact transmission') is incorporated in the model through a non-contact transmission rate that a

In [45]:
bert_results[bert_results['query']==trans_query][:3].style.set_properties(**{'font-size': '9pt'})

,query,text,model,sim
70,Ways of covid19 transmission,in COVID-19 transmission. Our dynamic modeling approach can be applied widely and will 33 provide a new fillip to infectious disease management strategies worldwide.,SBERT,0.927225
71,Ways of covid19 transmission,"In that unique context of scarce resources of both surgery and newest systemic therapies, IO should gain a leading role to cure or at least to control HCCs. We are used to recommend and follow evidence-based medicine in IO but in the COVID-19 context such data are not yet available. The common medical sense must rather apply and one rule should guide us: prevent any loss of chance for the patient. That being said, our responsibility is to maintain high-quality treatments within short delays and with spared human and hospital resources. The COVID-19",SBERT,0.880791
72,Ways of covid19 transmission," Diffusion of COVID19. Number of infected from 17 March, 2020 to April 2020 (Ministero della Salute, 2020).",SBERT,0.869444


### 'Basic reproductive number of covid19'

In [46]:
repr_query = 'Basic reproductive number of covid19'

In [21]:
repr_res = run_query_on_models(repr_query, models, model_names)
repr_res.style.set_properties(**{'font-size': '9pt'}).apply(color_rows, axis=1)

100%|██████████| 1/1 [00:00<00:00, 576.46it/s]


,query,text,model,sim
0,Basic reproductive number of covid19,"This article investigates the change in the basic reproduction number and controlled reproduction number since the outbreak of COVID-19. We have found that the estimated controlled reproduction numbers in all different regions are significantly smaller compared with the basic reproduction numbers , which indicates that the containment measures carried out by Chinese government was effective and efficient.",tf_idf,0.604279
1,Basic reproductive number of covid19,"The basic reproduction number is model-based, largely depends on the epidemiological setting, and is the most important parameter to determine intrinsic transmissibility. The early outbreak data largely follow exponential growth. Different models based on the clinical progression of the disease were devised to estimate the basic reproduction number. In the early stages of COVID-19, the pandemic doubled in size every 7.4 days, and the basic reproduction number was estimated to be 2.2 [7] . Another study estimated the basic reproduction number as ranging from 2.24 to 3.58 [11] . However, a deterministic compartmental model based on the likelihood and a model analysis showed that the control reproduction number may be as high as 6.47 [12] . The authors noted that this basic reproduction number was higher because the estimate accounts for 3-4 generations of viral transmission and intensive social contacts. The basic reproduction number estimated by the majority of studies ranges from 2.24 to 3.58 [13] , which is slightly higher than that of SARS.",tf_idf,0.583680
2,Basic reproductive number of covid19,"Studies such as [3] show that the effectiveness of social distancing is particularly great for pandemics with a basic reproductive number of R0 = 1.5 .. 2.5. For the Covid-19 pandemic caused by the SARS-CoV-2 virus, a basic reproduction number R0 = 2.6 is estimated, with an uncertainty range from 1.5 to 3.5 [5] . The new infection rate of 28% (in Germany) and 34% (in the US) for 10 days of infectiousness, which is the basis for this study, corresponds approximately to a basic reproductive number R0 = 2.8 .. 3.4.",tf_idf,0.581665
3,Basic reproductive number of covid19,The per-contact transmission rate of COVID-19 was estimated assuming a basic reproduction number of R0 = 3. We assumed that,bm25,0.480876
4,Basic reproductive number of covid19,". The next-generation matrix will be used to derive an analytical expression for the basic reproduction number (R 0 ), for the compartmental model above. Calculating R 0 is a useful metric for assessing the transmission potential of an emerging COVID19 in Saudi Arabia.",bm25,0.432316
5,Basic reproductive number of covid19,"This article investigates the change in the basic reproduction number and controlled reproduction number since the outbreak of COVID-19. We have found that the estimated controlled reproduction numbers in all different regions are significantly smaller compared with the basic reproduction numbers , which indicates that the containment measures carried out by Chinese government was effective and efficient.",bm25,0.422077
6,Basic reproductive number of covid19,"• R 0 and R e : The basic reproduction number and the effective reproduction number of COVID-19 (for territory i). The basic reproduction number is defined as the number of cases one infected person generates on average over the course of its infectious period, in an otherwise uninfected population and without special control measures.",w2v,0.840156
7,Basic reproductive number of covid19,"This article investigates the change in the basic reproduction number and controlled reproduction number since the outbreak of COVID-19. We have found that the estimated controlled reproduction numbers in all different regions are significantly smaller compared with the basic reproduction numbers , which indicates that the containment measures carried out by Chinese government was effective and efficient.",w2v,0.838988


In [47]:
bert_results[bert_results['query']==repr_query][:3].style.set_properties(**{'font-size': '9pt'})

,query,text,model,sim
0,Basic reproductive number of covid19,"Following [2, 3] , we assumed that number of secondary cases associated with a primary COVID-19 case follows a negative binomial (NB) distribution, with means R0 and dispersion parameter k [3] . Here, the R0 is the basic reproduction number of COVID-19. The k measures the likelihood of occurrence of super-spreading events (or other factors) which could vary the growth rate. If k is smaller than 1, these data indicate occurrence of super-spreading events (larger proportion of 'super-spreaders' or 'dead-ends' in infected individuals) or some other factors which could vary the growth rate; otherwise if k is larger than 1, these data do not indicate occurrence of super-spreading events [4] .",SBERT,0.816147
1,Basic reproductive number of covid19,"Countries confronting the COVID-19 pandemic are implementing different social distancing strategies. We evaluated the impact of small-area lockdowns in Chile, aimed to reduce viral transmission while minimizing the population disrupted. The effectiveness of this intervention on the outbreak control is unknown. A natural experiment assessing the impact of small-area lockdowns between February 15th and April 25th, 2020. We used mobility data and official governmental reports to compare regions with smallarea lockdowns versus regions without. The primary outcome was the mean difference in the effective reproductive number (Re) of COVID-19. Secondary outcomes were changes in mobility indicators. We used quasi-experimental methods for the analysis and examined the impact of other concurrent public health interventions to disentangle their effects. Small-area lockdown produced a sizable reduction in human mobility, equivalent to an 11.4% reduction (95%CI -14.4% to -8.38%) in public transport and similar effects in other mobility indicators. Ten days after implementation, the small-area lockdown produced a reduction of the effective reproductive number (Re) of 0.86 (95%CI -1.70 to -0.02). School and university closures, implemented earlier, led to a 40% reduction in urban mobility. Closure of educational institutions resulted in an even greater Re reduction compared with small-area lockdowns. Small-area lockdowns produced a reduction in mobility and viral transmission, but the effects were smaller than the early closures of schools and universities. Small-area lockdowns may have a relevant supporting role in reducing SARS-CoV-2 transmission and could be useful for countries considering scaling-down stricter social distancing interventions.",SBERT,0.808927
2,Basic reproductive number of covid19,"COVID-19 is caused by the virus SARS-CoV-2. Clinical manifestations of the disease resemble those of severe acute respiratory syndrome (SARS). The mortality rate is lower than in SARS (Zhou, Yu et al. 2020 ) but the incidence rate is significantly higher. The current COVID-19 epidemic differs in several aspects from the previous one caused by SARS, which was finally extinguished (Cossarizza, De Biasi et al. 2020 , Kucharski, Russell et al. 2020 , Kuniya 2020 , Leung 2020 , Mizumoto, Kagaya et al. 2020 , Parodi and Liu 2020 . First, COVID-19 has a higher basic reproductive number, R 0 , than SARS. Second, in contrast to SARS, causative pathogen transmission in COVID-19 starts before the end of the incubation stage of the disease. Third, unlike SARS, many (and possibly most) cases of COVID-19 are asymptomatic, but they are accompanied by a spread of causative pathogen. These features of COVID-19 lower the optimism for the prospect that the current epidemic will be successfully controlled.",SBERT,0.800297
